Author: Sophie-Christine Porak / ChatGPT

This script will count the number of tiles per protein segment therefore serving as an estimate of overall diversity at different protein regions. 
The absolute tile counts are being represented as a plot, where on the x-axis we have the position of the protein (amino acid position) and on the 
y-axis we have the absolute count of tiles for that position.

As the input is a nucleotide sequence, it needs to be translated to represent the amino acid sequence (or just divide by 3, I also know that each tile
represents 46 amino acids). 

Input: 
The headers should look the followingL 
>WFG38034.1_1_1
ATGGGGGATATCATCTCGTTT-nt sequence-

In [ ]:
from Bio import SeqIO #for parsing GenBank records
from Bio import Entrez #for accessing NCBI databases
from collections import defaultdict #convenient nested dictionary
import re #regular expression to parse headers
import csv
import os 

target_organism = "mammarenavirus lassaense"

Entrez.email = "sophie.porak@ucsf.edu" #required for ncbi access

fasta_file = '/Users/sophieporak/Library/CloudStorage/Box-Box/DeRisi/lassa fever project/data processing/mapping diversity to crystal structure/0.96_tiling_out_nt_tiles_cp_cleaned_no_linkers_only_accession_with_tile_number_052925.fasta'
base_output_dir = '/Users/sophieporak/Desktop/tile_diversity_060325'

#ensure output dir exists
os.makedirs(base_output_dir, exist_ok=True)

output_csv = os.path.join(base_output_dir,"tile_data_with_sequence.csv")
counts_csv = os.path.join(base_output_dir, "tile_position_counts.csv")
skipped_log = os.path.join(base_output_dir, "skipped_headers.log")

#store lookup results to avoid redundant API calls - this serves to cache results so each NCBI accession is fetched only once
accession_lookup = {}
#create a dictionary {(protein, organism): {tile_position: count}} 
tile_counts = defaultdict(lambda: defaultdict(int))
tile_rows = []
skipped_headers = []


#-------- FUNCTIONS --------

#function to fetch protein name and organism from NCBI
def fetch_info(accession):
    if accession in accession_lookup:
        #if accession already fetched it returns from the cache
        return accession_lookup[accession]

    try: 
        #downloads and parses the GenBank record from NCBI
        handle = Entrez.efetch(db="protein", id=accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()

        organism = record.annotations.get("organism", "unknown") #get the organism name
        protein_name = "unknown"

        #first trying to find product in CDS feature 
        for feature in record.features:
            if feature.type == "CDS" and "product" in feature.qualifiers:
                protein_name = feature.qualifiers["product"][0]
                break
        
        else:
            #if no CDS/product found, look in the protein feature
            for feature in record.features:
                if feature.type == "Protein" and "product" in feature.qualifiers:
                    protein_name = feature.qualifiers["product"][0]
                    break

        accession_lookup[accession] = (protein_name, organism)
        return protein_name, organism

    except Exception as e:
        print(f"Failed to fetch {accession}: {e}")
        accession_lookup[accession] = ("unknown", "unknown")
        return "unknown", "unknown"



# Process FASTA file
for record in SeqIO.parse(fasta_file, "fasta"):
    header = record.id
    sequence = str(record.seq)

    match = re.match(r"([\w.]+)_(\d+)_(\d+)", header)
    if not match:
        print(f"Skipped header: {header}")
        skipped_headers.append(header)
        continue

    accession, tile_pos, _ = match.groups()
    tile_pos = int(tile_pos)

    protein, organism = fetch_info(accession)

    # Store full data row
    tile_rows.append([accession, tile_pos, protein, organism, sequence])

    # Count occurrences
    tile_counts[(protein, organism)][tile_pos] += 1

if skipped_headers:
    with open(skipped_log, "w") as f:
        for header in skipped_headers:
            f.write(header + "\n")

# Write full tile data with sequence
with open(output_csv, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Accession", "Tile Position", "Protein", "Organism", "Sequence"])
    writer.writerows(tile_rows)

# Write summary count table
with open(counts_csv, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Protein", "Organism", "Tile Position", "Tile Count"])

    for (protein, organism), positions in tile_counts.items():
        for pos, count in sorted(positions.items()):
            writer.writerow([protein, organism, pos, count])

KeyboardInterrupt: 